In [1]:
import tensorflow as tf
q=tf.FIFOQueue(2,"int32")
init=q.enqueue_many(([0,10],))
X=q.dequeue()
y=X+1
q_inc=q.enqueue([y])
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    init.run()#一定要先初始化队列，不然运行不了，会卡住
    for _ in range(5):
        v,_=sess.run([X,q_inc])
        print(v)

0
10
1
11
2


In [3]:
import tensorflow as tf
import numpy as np
import threading
import time
def MyLoop(coord,worker_id):
    while(not coord.should_stop()):
        if(np.random.rand()<0.1):
            print("Stop from id:%d\n" % worker_id,coord.request_stop())
        else:
            print("Working on id:%d\n" % worker_id,time.sleep(1))
coord=tf.train.Coordinator()

threads=[threading.Thread(target=MyLoop,args=(coord,i,)) for i in range(5)]
for t in threads:
    t.start()
coord.join(threads)

Working on id:0
Working on id:3
Working on id:2
Working on id:4
 Working on id:1
  None None None
None
None


Working on id:3
Working on id:2
 None 
None
Working on id:4
 NoneWorking on id:1
Working on id:0

  NoneNone

Working on id:2
 Working on id:3
None
 None
Working on id:4
 None
Working on id:1
Working on id:0
  NoneNone

Working on id:2
 NoneWorking on id:3

 None
Stop from id:3
 None
Working on id:4
 None
Working on id:0
 None
Working on id:1
 None
Working on id:2
 None


In [7]:
import tensorflow as tf
queue=tf.FIFOQueue(100,"float")#队列中最多有100个元素
enqueue_op=queue.enqueue([tf.random_normal([1])])
qr=tf.train.QueueRunner(queue,[enqueue_op]*5)
tf.train.add_queue_runner(qr)
out_tensor=queue.dequeue()
with tf.Session() as sess:
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess,coord)
    for _ in range(3):
        print(sess.run(out_tensor)[0])
    coord.request_stop()
    coord.join(threads)
    
    

-1.64617
-1.12697
1.05122


In [82]:
import tensorflow as tf
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
#图片路径
cwd='E://aa//'
#文件路径
filepath='E://aa//'
#保存图片个数
bestnum=10
#第几个图片
num=0
#第几个TFRecord文件
recordfilenum=0

#类别
classes={'husky','chihuahua'}
#tfrecord格式文件名
tfrecordfilename=("data.tfrecords-%.3d" % recordfilenum)
writer=tf.python_io.TFRecordWriter(filepath+tfrecordfilename)
#类别和路径
for index,name in enumerate(classes):
    print(index)
    print(name)
    class_path=cwd+name+'\\'
    for img_name in os.listdir(class_path):
        num+=1
        if(num>bestnum):
            num=1
            recordfilenum=recordfilenum+1
            tfrecordfilename=("data.tfrecords-%.3d" % recordfilenum)
            writer=tf.python_io.TFRecordWriter(filepath+tfrecordfilename)
        img_path=class_path+img_name
        img=Image.open(img_path)
        img_raw=img.tobytes()
        examle=tf.train.Example(
        features=tf.train.Features(feature={
            "label":tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
            "img_raw":tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
        }))
        writer.write(examle.SerializeToString())
    writer.close()

0
chihuahua
1
husky


In [74]:
filename_queue = tf.train.string_input_producer(["data.tfrecords-000"]) #读入流中

In [85]:
import numpy as np
reader=tf.TFRecordReader()
_,serialized_example=reader.read(filename_queue)
features=tf.parse_single_example(serialized_example,
                                 features={
                                     'label':tf.FixedLenFeature([],tf.int64),
                                     'img_raw':tf.FixedLenFeature([],tf.string),
                                 })
image=tf.decode_raw(features['img_raw'],tf.uint8)
#image=tf.reshape(image,[128,128,3])
label=tf.cast(features['label'],tf.int32)
with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)
    
    for i in range(20):
        example,l=sess.run([image,label])
        img = np.fromstring(image, dtype=np.float32) 
        #img=Image.fromarray(example,'RGB')
        #img=tf.reshape(img,[256,256,3])
        img.save(cwd+str(i)+'_Label_'+str(i)+'.jpg')
        print(example,l)
    coord.request_stop()
    coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.FailedPreconditionError'>, Attempting to use uninitialized value matching_filenames
	 [[Node: matching_filenames/read = Identity[T=DT_STRING, _class=["loc:@matching_filenames"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](matching_filenames)]]


TypeError: a bytes-like object is required, not 'Tensor'